# Laboratorio 9: AdaBoost y contenedores

**Duración:** 2 horas  
**Formato:** Competencia con contenedores  

---

## Portada del equipo

**Integrantes:**
- Nombre 1 (Usuario GitHub)
- Nombre 2 (Usuario GitHub)
- Nombre 3 (Usuario GitHub)


**Fecha de entrega:**  
__/__/____

## Elemento 1 - Implementación desde cero de AdaBoost

### Elemento 1 - Preguntas teóricas

## Elemento 2 - Comparativa con scikit-learn

### Elemento 2 - Preguntas teóricas

## Elemento 3 - API, contenedor Docker y automatización con Makefile

### Elemento 3 - Preguntas teóricas

## Conclusiones de la práctica